In [51]:
%reload_ext autoreload

import os
import sys

import gnn4itk_cf.core
print(gnn4itk_cf.core.__file__)

import numpy as np
import pandas as pd
import scipy.sparse as sps
import yaml
from itertools import chain, product, combinations
import torch

from time import time as tt
from tqdm import tqdm
import wandb
from gnn4itk_cf.core.infer_stage import infer
from gnn4itk_cf.core.eval_stage import evaluate

from gnn4itk_cf.stages.data_reading.models.trackml_utils import *

from gnn4itk_cf.stages.data_reading.data_reading_stage import EventReader
from gnn4itk_cf.stages.data_reading.models.trackml_reader import TrackMLReader

from gnn4itk_cf.stages.graph_construction.models.metric_learning import MetricLearning
from gnn4itk_cf.stages.edge_classifier.models.filter import Filter
from gnn4itk_cf.stages.edge_classifier import InteractionGNN

from gnn4itk_cf.stages.graph_construction.utils import handle_weighting
from gnn4itk_cf.stages.graph_construction.models.utils import graph_intersection, build_edges
from gnn4itk_cf.stages.graph_construction.utils import *

from gnn4itk_cf.stages.track_building import utils 
from torch_geometric.utils import to_scipy_sparse_matrix

/global/homes/a/alazar/acorn/gnn4itk_cf/core/__init__.py


In [52]:
config_dr = "../../examples/Example_1/data_reader.yaml"
config_mm = "../../examples/Example_1/module_map_infer.yaml"
configGnn = "../../examples/Example_1/gnn_infer.yaml"
configGnn_eval = "../../examples/Example_1/gnn_eval.yaml"
config_tbi = "../../examples/Example_1/track_building_infer.yaml"
config_tbe = "../../examples/Example_1/track_building_eval.yaml"

In [53]:
#infer(config_dr )
#infer(config_mm)
infer(configGnn,configGnn['stage_dir']+'artifacts/best_3.ckpt')
#evaluate(configGnn_eval,None)
#infer(config_tbi)
evaluate(config_tbe,config_gnn['stage_dir']+'artifacts/best_3.ckpt')

TypeError: string indices must be integers

In [54]:
with open("../../examples/Example_2/metric_learning_train.yaml", "r") as f:
    config_ml = yaml.load(f, Loader=yaml.FullLoader)
model_ml = MetricLearning(config_ml)
with open("../../examples/Example_1/gnn_train.yaml", "r") as f:
    config_gnn = yaml.load(f, Loader=yaml.FullLoader)
model_gnn = InteractionGNN(config_gnn)

run = wandb.init(project=model_gnn.hparams["project"], entity='gnnproject')

config_tbe = yaml.safe_load(open("../../examples/Example_1/track_building_eval.yaml", "r"))
print(config_gnn['stage_dir']+'artifacts')

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


/scratch/cf/Example_1/gnn/artifacts


In [55]:
#model_ml.setup(stage="predict")
#dataloaders = model_ml.predict_dataloader()
model_ml = MetricLearning.load_from_checkpoint(config_ml['stage_dir']+'artifacts/best.ckpt')
model_gnn.setup('predict')
dataloaders = model_gnn.predict_dataloader()
model_gnn = InteractionGNN.load_from_checkpoint(config_gnn['stage_dir']+'artifacts/best-v3.ckpt')
#model_gnn = InteractionGNN.load_from_checkpoint('../../model_store/Example_1/gnn.ckpt', config=config_gnn)

Lightning automatically upgraded your loaded checkpoint from v1.8.6 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../../scratch/cf/Example_2/metric_learning/artifacts/best.ckpt`


/global/homes/a/alazar/acorn/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [particle_id] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/homes/a/alazar/acorn/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [nhits] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/homes/a/alazar/acorn/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [primary] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/homes/a/alazar/acorn/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [pdgId] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/homes/a/alazar/acorn/gnn4itk_cf/utils/loading_utils.py:78: UserWarning: OPTIONAL feature [ghost] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/global/homes/a/alazar/acorn/gnn4itk_cf/ut

Defining figures of merit


/global/homes/a/alazar/acorn/gnn4itk_cf/stages/edge_classifier/edge_classifier_stage.py:94: UserWarning: Failed to define figures of merit, due to logger unavailable
  warnings.warn(


In [ ]:
#infer(config_dr )
#infer(config_mm)
#print(config_gnn['stage_dir']+'artifacts/best_3.ckpt')
infer(configGnn,'/scratch/cf/Example_1/gnn/artifacts/best_3.ckpt')
#evaluate(configGnn_eval,None)
#infer(config_tbi)
#evaluate(config_tbe,config_gnn['stage_dir']+'artifacts/best_3.ckpt')

evaluate(config_tbe, None) #,'../../model_store/Example_1/gnn.ckpt')

TypeError: infer() takes 1 positional argument but 2 were given

In [56]:
def evaluate_labelled_graphs(graphset, config):
    all_y_truth, all_pt  = [], []
    evaluated_events = [
        utils.evaluate_labelled_graph(
            event,
            matching_fraction=config["matching_fraction"],
            matching_style=config["matching_style"],
            min_track_length=config["min_track_length"],
            min_particle_length=config["min_particle_length"],
        )
        for event in tqdm(graphset)
    ]
    evaluated_events = pd.concat(evaluated_events)

    particles = evaluated_events[evaluated_events["is_reconstructable"]]
    reconstructed_particles = particles[particles["is_reconstructed"] & particles["is_matchable"]]
    tracks = evaluated_events[evaluated_events["is_matchable"]]
    matched_tracks = tracks[tracks["is_matched"]]

    n_particles = len(particles.drop_duplicates(subset=['event_id', 'particle_id']))
    n_reconstructed_particles = len(reconstructed_particles.drop_duplicates(subset=['event_id', 'particle_id']))

    n_tracks = len(tracks.drop_duplicates(subset=['event_id', 'track_id']))
    n_matched_tracks = len(matched_tracks.drop_duplicates(subset=['event_id', 'track_id']))

    n_dup_reconstructed_particles = len(reconstructed_particles) - n_reconstructed_particles

    print(f"Number of reconstructed particles: {n_reconstructed_particles}")
    print(f"Number of particles: {n_particles}")
    print(f"Number of matched tracks: {n_matched_tracks}")
    print(f"Number of tracks: {n_tracks}")
    print(f"Number of duplicate reconstructed particles: {n_dup_reconstructed_particles}")   

    # Plot the results across pT and eta
    eff = n_reconstructed_particles / n_particles
    fake_rate = 1 - (n_matched_tracks / n_tracks)
    dup_rate = n_dup_reconstructed_particles / n_reconstructed_particles

    logging.info(f"Efficiency: {eff:.3f}")
    logging.info(f"Fake rate: {fake_rate:.3f}")
    logging.info(f"Duplication rate: {dup_rate:.3f}")
    print(f"Efficiency: {eff:.3f}")
    print(f"Fake rate: {fake_rate:.3f}")
    print(f"Duplication rate: {dup_rate:.3f}")

In [57]:
device ='cuda'
model_ml = model_ml.to("cuda")
model_gnn = model_gnn.to("cuda")
graphs = []
for batch_idx, batch in enumerate(dataloaders[2]):
    batch = batch.to("cuda")    
    gnn = model_gnn.shared_evaluation(batch,batch_idx)
    #print(gnn['output'].max(),gnn['output'].min())
    batch = gnn['batch']
    #model_gnn.log_metrics(gnn['output'],gnn['all_truth'],gnn['target_truth'],gnn['loss'])
    edge_mask = gnn['output'] > 0.8 #model_gnn.hparams['edge_cut'] # score_cut for evaluation
    
    # Get number of nodes
    if hasattr(batch, "num_nodes"):
        num_nodes = batch.num_nodes
    elif hasattr(batch, "x"):
        num_nodes = batch.x.size(0)
    elif hasattr(batch, "x_x"):
        num_nodes = batch.x_x.size(0)
    else:
        num_nodes = batch.edge_index.max().item() + 1
    # Convert to sparse scipy array
    sparse_edges = to_scipy_sparse_matrix(
        batch.edge_index[:, edge_mask], num_nodes=num_nodes
    )
    # Run connected components
    candidate_labels = sps.csgraph.connected_components(
        sparse_edges, directed=False, return_labels=True
    )
    batch.labels = torch.from_numpy(candidate_labels[1]).long()
    graphs.append(batch.to('cpu'))

evaluate_labelled_graphs(graphs, config_tbe)

tensor(0.9999, device='cuda:0') tensor(3.4809e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5200e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5047e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5351e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5390e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5306e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5325e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.4633e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5302e-05, device='cuda:0')
tensor(0.9999, device='cuda:0') tensor(3.5531e-05, device='cuda:0')


100%|██████████| 10/10 [00:01<00:00,  8.26it/s]


Number of reconstructed particles: 14613
Number of particles: 16081
Number of matched tracks: 16259
Number of tracks: 16377
Number of duplicate reconstructed particles: 1638
Efficiency: 0.909
Fake rate: 0.007
Duplication rate: 0.112
